In [1]:
! git clone https://github.com/Duckkyy/3rd_year_experiment.git

Cloning into '3rd_year_experiment'...
remote: Enumerating objects: 37126, done.
remote: Counting objects: 100% (2230/2230), done.
remote: Compressing objects: 100% (2100/2100), done.
remote: Total 37126 (delta 120), reused 2229 (delta 119), pack-reused 34896
Receiving objects: 100% (37126/37126), 913.52 MiB | 29.01 MiB/s, done.
Resolving deltas: 100% (14164/14164), done.
Updating files: 100% (38760/38760), done.


In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

2023-01-04 09:05:50.178415: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-04 09:05:50.237338: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-04 09:05:50.354318: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-04 09:05:50.355082: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

'/device:GPU:0'

In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [4]:
#Load libraries
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib


# New Section

In [5]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [9]:
import subprocess
subprocess.run(['ls'])
os.path.isdir('3rd_year_experiment/emotions/data/train')

3rd_year_experiment
__notebook_source__.ipynb


True

In [10]:
#Dataloader
data_path = '3rd_year_experiment/emotions/data/'
#Path for training and testing directory
train_path = os.path.join(data_path, 'train')
val_path = os.path.join(data_path, 'val')

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
val_loader=DataLoader(
    torchvision.datasets.ImageFolder(val_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [14]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [15]:
print(classes)
print(device)

['1', '2', '3', '4', '5', '6', '7']
cuda:0


In [16]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=7):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output

In [17]:
model=ConvNet(num_classes=7).to(device)

In [18]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()
num_epochs=20

In [30]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/*/*/*.jpg'))
val_count=len(glob.glob(val_path+'/*/*/*.jpg'))
print(train_count,val_count)

1504 83


In [31]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(val_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/val_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(23.8747) Train Accuracy: 0.2214095744680851 Test Accuracy: 0.18072289156626506
Epoch: 1 Train Loss: tensor(9.1486) Train Accuracy: 0.32978723404255317 Test Accuracy: 0.24096385542168675
Epoch: 2 Train Loss: tensor(4.8999) Train Accuracy: 0.4341755319148936 Test Accuracy: 0.20481927710843373
Epoch: 3 Train Loss: tensor(2.4459) Train Accuracy: 0.574468085106383 Test Accuracy: 0.25301204819277107
Epoch: 4 Train Loss: tensor(1.3772) Train Accuracy: 0.694813829787234 Test Accuracy: 0.24096385542168675
Epoch: 5 Train Loss: tensor(1.4123) Train Accuracy: 0.7121010638297872 Test Accuracy: 0.1686746987951807
Epoch: 6 Train Loss: tensor(1.4542) Train Accuracy: 0.7333776595744681 Test Accuracy: 0.20481927710843373
Epoch: 7 Train Loss: tensor(1.1048) Train Accuracy: 0.7799202127659575 Test Accuracy: 0.27710843373493976
Epoch: 8 Train Loss: tensor(0.8333) Train Accuracy: 0.8324468085106383 Test Accuracy: 0.1686746987951807
Epoch: 9 Train Loss: tensor(0.5443) Train Accura

In [32]:
#prediction function
def prediction(img_path,transformer, model):
    image=Image.open(img_path)
    image_tensor=transformer(image).float()
    image_tensor=image_tensor.unsqueeze_(0)
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    output=model(input)
    
    index=output.data.numpy().argmax()
    pred=classes[index]
    
    return pred

In [33]:
checkpoint=torch.load('best_checkpoint.model')
model=ConvNet(num_classes=7)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=7, bias=True)
)

In [36]:
from tqdm.auto import tqdm
import pandas as pd
from PIL import Image

test_path = "3rd_year_experiment/emotions/data/test"
label_path = "3rd_year_experiment/emotions/Labels"
image_path = '3rd_year_experiment/emotions/datasets'

test_label_path = os.path.join(label_path, "ground_truth_test.csv")

test_df = pd.read_csv(test_label_path)
iterow = list(test_df.iterrows())

for idx, row in tqdm(iterow):
        test_image_path = os.path.join(image_path, row["image"])

        pred = prediction(test_image_path, transformer, model)

        test_df.loc[test_df["image"]==row["image"], "label"] = pred

test_df = test_df.astype({'label': 'int32'})

test_df.to_csv("sample_submission.csv", index=False)

  0%|          | 0/393 [00:00<?, ?it/s]